In [ ]:
from titan.tcrux.services import tcrux_multi_tasking

Run the service to train the logistic model for banking problem.

The output will be a Hashmap with score, recall, precision, and saved_model

In [4]:
params_train = [
    {
        "task": "loading",
        "action": "load_dataframe",
        "kwargs": {"input_arrow": "tests/resources/logistic.arrow"},
    },
    {
        "task": "regression",
        "action": "logistic_regression",
        "kwargs": {
            "target": "deposit",
            "stratify_column": "loan",
            "model_saved_path": "logistic_model.pkl",
            "processors": {
                "numerical_columns": [
                    "age",
                    "balance",
                    "day",
                    "campaign",
                    "pdays",
                    "previous",
                    "duration",
                ],
                "categorical_columns": [
                    "job",
                    "marital",
                    "education",
                    "default",
                    "housing",
                    "loan",
                ],
            },
        },
    },
]

In [5]:
result = tcrux_multi_tasking(params_train)

In [6]:
result

{'score': 0.7819077474249888,
 'precision': 0.7850834151128557,
 'recall': 0.7490636704119851,
 'model_saved_path': 'logistic_model.pkl'}

Now we can predict model from saved model. The input data has the form.

In [13]:
# read data from arrow file
import polars as pl

In [15]:
test_data = pl.read_ipc("tests/resources/logistic_test.arrow")

Could not memory_map compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.


The test data has the same form as original data set, except the deposit column does not exist.

In [17]:
test_data

age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
i64,str,str,str,str,i64,str,str,str,i64,str,i64,i64,i64,i64,str
34,"""blue-collar""","""divorced""","""secondary""","""no""",0,"""yes""","""no""","""cellular""",5,"""may""",150,1,-1,0,"""unknown"""
36,"""blue-collar""","""married""","""secondary""","""no""",29,"""yes""","""yes""","""cellular""",31,"""jul""",345,1,-1,0,"""unknown"""
31,"""blue-collar""","""married""","""secondary""","""no""",435,"""yes""","""no""","""unknown""",21,"""may""",204,1,-1,0,"""unknown"""
36,"""blue-collar""","""divorced""","""secondary""","""no""",7051,"""no""","""yes""","""cellular""",15,"""jul""",106,13,-1,0,"""unknown"""
31,"""admin.""","""married""","""secondary""","""no""",8781,"""yes""","""no""","""cellular""",14,"""may""",680,1,177,1,"""success"""
66,"""housemaid""","""married""","""primary""","""no""",40,"""no""","""no""","""telephone""",14,"""oct""",290,2,-1,0,"""unknown"""
26,"""blue-collar""","""married""","""secondary""","""no""",557,"""yes""","""yes""","""unknown""",27,"""may""",282,2,-1,0,"""unknown"""
39,"""services""","""married""","""secondary""","""no""",20928,"""no""","""no""","""cellular""",14,"""may""",1166,1,352,1,"""other"""
25,"""blue-collar""","""married""","""secondary""","""no""",101,"""no""","""yes""","""cellular""",8,"""jul""",460,3,-1,0,"""unknown"""


In [18]:
params_test = [
    {
        "task": "loading",
        "action": "load_dataframe",
        "kwargs": {"input_arrow": "tests/resources/logistic_test.arrow"},
    },
    {
        "task": "regression",
        "action": "logistic_predict",
        "kwargs": {
            "model_saved_path": "logistic_model.pkl",
            "target": "predicted_deposit",
        },
    },
    {
        "task": "saving",
        "action": "write_dataframe",
        "kwargs": {
            "output_arrow": "predicted.arrow",
            "batch_size": 20,
            "reformat_string": True,
        },
    },
]

In [19]:
tcrux_multi_tasking(params_test)

'predicted.arrow'

In [20]:
pl.read_ipc("predicted.arrow")

age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,predicted_deposit
i64,str,str,str,str,i64,str,str,str,i64,str,i64,i64,i64,i64,str,str
34,"""blue-collar""","""divorced""","""secondary""","""no""",0,"""yes""","""no""","""cellular""",5,"""may""",150,1,-1,0,"""unknown""","""no"""
36,"""blue-collar""","""married""","""secondary""","""no""",29,"""yes""","""yes""","""cellular""",31,"""jul""",345,1,-1,0,"""unknown""","""no"""
31,"""blue-collar""","""married""","""secondary""","""no""",435,"""yes""","""no""","""unknown""",21,"""may""",204,1,-1,0,"""unknown""","""no"""
36,"""blue-collar""","""divorced""","""secondary""","""no""",7051,"""no""","""yes""","""cellular""",15,"""jul""",106,13,-1,0,"""unknown""","""no"""
31,"""admin.""","""married""","""secondary""","""no""",8781,"""yes""","""no""","""cellular""",14,"""may""",680,1,177,1,"""success""","""yes"""
66,"""housemaid""","""married""","""primary""","""no""",40,"""no""","""no""","""telephone""",14,"""oct""",290,2,-1,0,"""unknown""","""no"""
26,"""blue-collar""","""married""","""secondary""","""no""",557,"""yes""","""yes""","""unknown""",27,"""may""",282,2,-1,0,"""unknown""","""no"""
39,"""services""","""married""","""secondary""","""no""",20928,"""no""","""no""","""cellular""",14,"""may""",1166,1,352,1,"""other""","""yes"""
25,"""blue-collar""","""married""","""secondary""","""no""",101,"""no""","""yes""","""cellular""",8,"""jul""",460,3,-1,0,"""unknown""","""no"""
